# ASL Training Convolutional Neural Network

ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

For the training images, we used an opensource [dataset] (https://www.kaggle.com/grassknoted/asl-alphabet). It is a series of folders containing images of letters in english alphabets along with couple of special characters. The notion of the training is to learn the weights to correctly identify the letter that corresponds to the sign.

In [1]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
#from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
imageSize =64
train_dir = "../asl-alphabet/asl_alphabet_train/"
test_dir =  "../asl-alphabet/asl_alphabet_test/"

## 1. Reading data

From the selected input files, read the data and find the X, y for the training. X will be a image input and y its respective class, which we will give an one-hot encoding for.

In [4]:
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['A']:
                label = 0
            elif folderName in ['B']:
                label = 1
            elif folderName in ['C']:
                label = 2
            elif folderName in ['D']:
                label = 3
            elif folderName in ['E']:
                label = 4
            elif folderName in ['F']:
                label = 5
            elif folderName in ['G']:
                label = 6
            elif folderName in ['H']:
                label = 7
            elif folderName in ['I']:
                label = 8
            elif folderName in ['J']:
                label = 9
            elif folderName in ['K']:
                label = 10
            elif folderName in ['L']:
                label = 11
            elif folderName in ['M']:
                label = 12
            elif folderName in ['N']:
                label = 13
            elif folderName in ['O']:
                label = 14
            elif folderName in ['P']:
                label = 15
            elif folderName in ['Q']:
                label = 16
            elif folderName in ['R']:
                label = 17
            elif folderName in ['S']:
                label = 18
            elif folderName in ['T']:
                label = 19
            elif folderName in ['U']:
                label = 20
            elif folderName in ['V']:
                label = 21
            elif folderName in ['W']:
                label = 22
            elif folderName in ['X']:
                label = 23
            elif folderName in ['Y']:
                label = 24
            elif folderName in ['Z']:
                label = 25
            elif folderName in ['del']:
                label = 26
            elif folderName in ['nothing']:
                label = 27
            elif folderName in ['space']:
                label = 28           
            else:
                label = 29
            for image_filename in os.listdir(folder + folderName):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_arr = np.resize(np.asarray(img_file), (imageSize, imageSize, 3))
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
X_train, y_train = get_data(train_dir) 

NameError: name 'os' is not defined

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2) 

NameError: name 'X_train' is not defined

In [6]:
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(y_train, num_classes = 29)
y_testHot = to_categorical(y_test, num_classes = 29)

ModuleNotFoundError: No module named 'keras'

In [7]:
#shuffling the data, so that overfitting does not occur
from sklearn.utils import shuffle
X_train, y_trainHot = shuffle(X_train, y_trainHot, random_state=13)
X_test, y_testHot = shuffle(X_test, y_testHot, random_state=13)
X_train = X_train[:30000]
X_test = X_test[:30000]
y_trainHot = y_trainHot[:30000]
y_testHot = y_testHot[:30000]

NameError: name 'X_train' is not defined

## 2. Building the model

Using [keras] (www.keras.io), a high level neural network API, we are building the model with ease. We have added dropouts to prevent overfitting and make the model robust. Once the model is built, it is training using Adam Optimizer for n epochs (n can vary). We need to save the model and weights so that we can reuse them.

In [2]:

#model building
target_dims =(64,64,3)
my_model = Sequential()
my_model.add(Conv2D(50, kernel_size=4, strides=1, activation='relu', input_shape=target_dims))
my_model.add(Conv2D(50, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
my_model.add(Flatten())
my_model.add(Dropout(0.5))
my_model.add(Dense(512, activation='relu'))
my_model.add(Dense(29, activation='softmax'))
#back propogation initialization
my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

NameError: name 'Sequential' is not defined

In [3]:
my_model.fit(X_train,y_trainHot, epochs=200, batch_size=1024,verbose=0) #change verbose to 1 to visualize the learning

NameError: name 'my_model' is not defined

In [4]:
#Saving the model and weights to a file

model = my_model
model_json = model.to_json()
with open("../models/model_self.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../models/model_self.h5")
print("Saved model to disk")

NameError: name 'my_model' is not defined

## 3. Testing the saved model

Once the model is saved we can test it with a set of images that the models has not seen

In [5]:
data_dir = "input/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64, 64, 3) # add channel for RGB
n_classes = 29
val_frac = 0.1
batch_size = 64

class ASLPredictor:
    def __init__(self):
        # load json and create model
        json_file = open('models/self/model_self.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        # load weights into new model
        self.model.load_weights("models/self/model_self.h5")
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
        self.labels = {'0': 'A', '1': 'B', '2': 'C', '3': 'D', '4': 'E', '5': 'F', '6': 'G', '7': 'H', '8': 'I','9': 'J', '10': 'K', '11': 'L', '12': 'M', '13': 'N', '14': 'O', '15': 'P', '16': 'Q', '17': 'R', '18': 'S', '19': 'T', '20': 'U', '21': 'V', '22': 'W', '23': 'X', '24': 'W', '25': 'Z', '26': 'de', '27': 'nothing', '28': ' '}
        print("Loaded model from disk")
    
    def predict(self):
        image_dir = "../input/asl_alphabet_test/test"  
        for image_files in os.listdir(image_dir):
           image = cv2.imread(image_dir + '/' + image_files)
           if image is not None:
            img = np.asarray(np.resize(image,(1,64,64,3)))
            probabilities = self.model.predict(img)
            y_classes = probabilities.argmax(axis=-1)
            print(image_files + "-> "+ str(y_classes))

    def predictor(self,image):
        #converting the image into standard input size
        img = np.asarray(np.resize(image,(1,64,64,3)))
        probabilities = self.model.predict(img)
        #argmax accross the last layer to select a winner
        y_class = probabilities.argmax(axis=-1)[0]
        return str(self.labels[str(y_class)])
   

c = ASLPredictor()
c.predict()


FileNotFoundError: [Errno 2] No such file or directory: 'models/self/model_self.json'